My project will be to try and find the most gentirifed locations in NY, Toronto, and, if possible, Ottawa. Using the the same methods as in the previous task we can find the most popular venues for every neighbourhood. I plan to exclude all neighbourhoods in which at least three of the top 5 venues are not a coffee shop, a yoga studio, a gym or a restaraunt focusing on foreign food. The remaining neighbourhoods should be fairly similar, so I hope the clustering process will separate the neighbourhoods by the concentration of gentrification-indicating venues.

My goal is to find neighbourhoods which are in the process of being gentirified, but aren't quite there yet. The clients are the business owners looking to open cofee shops. To avoid too much competition, neighbourhoods with cofee shops as the first and second most popular venues will also be excluded.

I'll use existing data for NY and Toronto, for why fix what isn't broken. For Ottawa neighbourhoods I'm betting on Wikipedia.
[Neighbourhoods of Ottawa](https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Ottawa)